In [34]:
import pandas as pd

df = pd.read_csv('data/education/사설학원수.csv')
df.head(), df.shape, df['지역별'].unique()


(  지역별    시점  사설학원수
 0  서울  2003  11176
 1  서울  2004  11382
 2  서울  2005  11603
 3  서울  2006  11794
 4  서울  2007  11549,
 (348, 3),
 array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
        '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object))

In [35]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'사설학원수': 'mean'}).round(3)

    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime 형식으로 되어 있어야 함
    # 마지막 년도에 12월 데이터 추가
    last_year = monthly_df['시점'].dt.year.max()
    last_month = monthly_df['시점'].dt.month.max()
    
    # 12월 데이터가 없으면 추가
    if last_month < 12:
        new_row = pd.DataFrame({
            '사설학원수': [monthly_df['사설학원수'].iloc[-1]],  # 마지막 값을 사용
            '시점': [pd.Timestamp(year=last_year, month=12, day=1)],
            '지역별': [region]
        })
        monthly_df = pd.concat([monthly_df, new_row], ignore_index=True)

    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)

# '시점'을 문자열로 다시 포맷
final_monthly_df['시점'] = final_monthly_df['시점'].dt.strftime('%Y.%m')

# 사설학원수 형식 변경
final_monthly_df['사설학원수'] = final_monthly_df['사설학원수'].map(lambda x: f"{x:.0f}")

# 데이터 저장
final_monthly_df.to_csv('data/education/사설학원수(월별).csv', index=False)

# 결과 확인
print(final_monthly_df.tail(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# 데이터 저장
final_monthly_df.to_csv('data/education/사설학원수(월별).csv', index=False)


           시점 사설학원수 지역별
3977  2022.02  2389  충북
3978  2022.03  2396  충북
3979  2022.04  2404  충북
3980  2022.05  2412  충북
3981  2022.06  2420  충북
3982  2022.07  2428  충북
3983  2022.08  2435  충북
3984  2022.09  2443  충북
3985  2022.10  2451  충북
3986  2022.11  2458  충북
3987  2022.12  2466  충북
3988  2023.01  2474  충북 (3989, 3) ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
